# Apredizagem por Regras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as srn
import plotly
import pickle
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import classification_report

# Base Risco de Crédito

## Pré-processamento

In [2]:
import pickle
with open('risco_credito.pkl', 'rb') as f:
    X_risco, y_risco = pickle.load(f)

In [3]:
print(X_risco.shape)
print(y_risco.shape)

(14, 4)
(14,)


## OneR

Pode-se usar o Weka explorer

# Base Credit Data

## Pré-processamento

In [4]:
with open('credit.pkl', 'rb') as f:
    X_credit_train, y_credit_train, X_credit_test, y_credit_test = pickle.load(f)

In [5]:
print(X_credit_train.shape, y_credit_train.shape)
print(X_credit_test.shape, y_credit_test.shape)

(1500, 3) (1500,)
(500, 3) (500,)


# Base Census

## Pré-processamento

In [6]:
with open ('census.pkl', 'rb') as f:
    X_census_train, y_census_train, X_census_test, y_census_test = pickle.load(f)

In [7]:
print(X_census_train.shape, y_census_train.shape)
print(X_census_test.shape, y_census_test.shape)

(27676, 108) (27676,)
(4885, 108) (4885,)


## OneR

Pode-se usar o Weka explorer